In [1]:
import pandas as pd
import datetime as dt

def correlation_check(Anlagenleistung, Netzverfügbarkeit, Bilanzieller_Ausgleich):
    
    #Leistungsdaten einlesen und zu einem Dataframe binden
    df = pd.DataFrame()
    for file in Anlagenleistung:
        df = pd.concat([df, pd.read_excel(file)], ignore_index = True)
        df = df.drop_duplicates()
    
    #Indexkorrektur
    index = list(range(df.shape[0]))
    df['Index'] = index
    df = df.set_index('Index')
    
    #Netzausfälle entfernen
    df1 = pd.read_excel(Netzverfügbarkeit, header = 9)
    n = 0
    rows_to_drop = []
    while n < df.shape[0]:
        m = 0
        while m < df1.shape[0]:
            if (df.loc[n, 'Timestamp'] >= df1.loc[m, 'Abschaltzeit'] and df.loc[n, 'Timestamp'] <= df1.loc[m, 'Freigabe zur Einspeisung von TenneT']):
                rows_to_drop.append(n)
                break
            elif (df.loc[n, 'Timestamp'] > df1.loc[m, 'Freigabe zur Einspeisung von TenneT'] and df.loc[n, 'Timestamp'] < df1.loc[m+1, 'Abschaltzeit']):
                break
            m += 1
        n += 1
    
    df = df.drop(rows_to_drop)
    print('Netzausfälle entfernt')

    #Indexkorrektur
    index = list(range(df.shape[0]))
    df['Index'] = index
    df = df.set_index('Index')

    ###########      REDISPATCH ENTFERNEN          ##################
    
    #Bilanziellen Ausgleich einlesen
    df2 = pd.DataFrame()
    for file in Bilanzieller_Ausgleich:
        df2 = pd.concat([df2, pd.read_excel(file, names = ['Zeitstempel', 'Bilanzieller Ausgleich [MW]', 'Bilanzieller Redispatch Global Tech I', 'Werte', 'Einheit'])], ignore_index = True)
        df2 = df2.drop_duplicates()

    #Indexkorrektur
    index = list(range(df2.shape[0]))
    df2['Index'] = index
    df2 = df2.set_index('Index')

    #Filtere Zeitpunkte zu welchen Redispatch stattgefunden hat
    n = 0
    rows_to_drop = []
    while n < df2.shape[0]:
        if df2.loc[n, 'Bilanzieller Ausgleich [MW]'] == 0:
            rows_to_drop.append(n)
        n += 1
        
    df2 = df2.drop(rows_to_drop)

    #Indexkorrektur
    index = list(range(df2.shape[0]))
    df2['Index'] = index
    df2 = df2.set_index('Index')

    #Redispatch Ende einfügen
    df2['Zeitstempel Ende'] = df2['Zeitstempel'] + dt.timedelta(minutes = 15)

    #Entferne Zeitpunkte innerhalb einer Redispatch Maßnahme
    n = 1
    rows_to_drop = []
    while n < df2.shape[0] - 1: 
        if (df2.loc[n-1, 'Zeitstempel Ende'] == df2.loc[n, 'Zeitstempel'] and df2.loc[n, 'Zeitstempel Ende'] == df2.loc[n + 1, 'Zeitstempel']):
            rows_to_drop.append(n)
        n += 1

    df2 = df2.drop(rows_to_drop)

    #Indexkorrektur
    index = list(range(df2.shape[0]))
    df2['Index'] = index
    df2 = df2.set_index('Index')

    #Redispatch entfernen
    n = 0
    rows_to_drop = []
    while n < df.shape[0]:
        m = 1
        while m < df2.shape[0]:
            if (df.loc[n, 'Timestamp'] >= df2.loc[m - 1, 'Zeitstempel'] and df.loc[n, 'Timestamp'] < df2.loc[m, 'Zeitstempel Ende'] ):
                rows_to_drop.append(n)
                break
            elif m < df2.shape[0] - 1: 
                if (df.loc[n, 'Timestamp'] >= df2.loc[m, 'Zeitstempel Ende'] and df.loc[n, 'Timestamp'] < df2.loc[m + 1, 'Zeitstempel']):
                    break
            m += 2
        n += 1

    df = df.drop(rows_to_drop)

    #Indexkorrektur
    index = list(range(df.shape[0]))
    df['Index'] = index
    df = df.set_index('Index')
    
    print('Redispatch entfernt')
    
    return df


In [2]:
Anlagenleistung = [
    '../../Multiversum/2024/01-2024/power/10-min-Mittelwerte Anlagenleistung.xlsx', 
    '../../Multiversum/2024/02-2024/power/10-min-Mittelwerte Anlagenleistung.xlsx',
    '../../Multiversum/2024/03-2024/power/10-min-Mittelwerte Anlagenleistung.xlsx',
    '../../Multiversum/2024/04-2024/power/10-min-Mittelwerte Anlagenleistung.xlsx',
    '../../Multiversum/2024/05-2024/power/10-min-Mittelwerte Anlagenleistung.xlsx',
    '../../Multiversum/2024/06-2024/Multiversum 10-min-Mittelwerte Anlagenleistung_2024-07-02_11.49.xlsx',
    '../../Multiversum/2024/07-2024/Multiversum 10-min-Mittelwerte Anlagenleistung_2024-08-22_14.36.xlsx',
    '../../Multiversum/2024/08-2024/power/Multiversum 10-min-Mittelwerte Anlagenleistung_2024-09-10_10.44.xlsx',
    '../../Multiversum/2024/09-2024/Multiversum 10-min-Mittelwerte Anlagenleistung_2024-09.xlsx',
    '../../Multiversum/2024/10-2024/10-min-Mittelwerte Anlagenleistung_2024-10.xlsx',
    '../../Multiversum/2024/11-2024/Multiversum 10-min-Mittelwerte Anlagenleistung_2024-11.xlsx'
]

Netzverfügbarkeit = 'Netzverfügbarkeiten.xlsx'


Bilanzieller_Ausgleich = [
    '../../Redispatch/Bilanzieller Ausgleich/01-2024/Bilanzieller Ausgleich Januar 2024 Danske.xlsx',
    '../../Redispatch/Bilanzieller Ausgleich/02-2024/Bilanzieller Ausgleich Februar 2024 Danske.xlsx',
    '../../Redispatch/Bilanzieller Ausgleich/03-2024/Bilanzieller Ausgleich März 2024 Danske.xlsx',
    '../../Redispatch/Bilanzieller Ausgleich/04-2024/Bilanzieller Ausgleich April 2024 Danske.xlsx',
    '../../Redispatch/Bilanzieller Ausgleich/05-2024/Bilanzieller Ausgleich Mai 2024 Danske.xlsx',
    '../../Redispatch/Bilanzieller Ausgleich/06-2024/Bilanzieller Ausgleich Juni 2024 Danske.xlsx',
    '../../Redispatch/Bilanzieller Ausgleich/07-2024/Bilanzieller Ausgleich Juli 2024 Danske.xlsx',
    '../../Redispatch/Bilanzieller Ausgleich/08-2024/Bilanzieller Ausgleich August 2024 Danske.xlsx',
    '../../Redispatch/Bilanzieller Ausgleich/09-2024/Bilanzieller Ausgleich September 2024 Danske.xlsx',
    '../../Redispatch/Bilanzieller Ausgleich/10-2024/Bilanzieller Ausgleich Oktober 2024 Danske.xlsx',
    '../../Redispatch/Bilanzieller Ausgleich/11-2024/Bilanzieller Ausgleich November 2024 Danske.xlsx'
]

In [173]:
#Bilanziellen Ausgleich einlesen
df2 = pd.DataFrame()

for file in Bilanzieller_Ausgleich:
    df2 = pd.concat([df2, pd.read_excel(file, names = ['Zeitstempel', 'Bilanzieller Ausgleich [MW]', 'Bilanzieller Redispatch Global Tech I', 'Werte', 'Einheit'])], ignore_index = True)
    df2 = df2.drop_duplicates()

In [174]:
#Indexkorrektur
index = list(range(df2.shape[0]))
df2['Index'] = index
df2 = df2.set_index('Index')

In [175]:
#Filtere Zeitpunkte zu welchen Redispatch stattgefunden hat
n = 0
rows_to_drop = []
while n < df2.shape[0]:
    if df2.loc[n, 'Bilanzieller Ausgleich [MW]'] == 0:
        rows_to_drop.append(n)
    n += 1


In [176]:
df2 = df2.drop(rows_to_drop)

In [177]:
#Indexkorrektur
index = list(range(df2.shape[0]))
df2['Index'] = index
df2 = df2.set_index('Index')

In [178]:
df2['Zeitstempel Ende'] = df2['Zeitstempel'] + dt.timedelta(minutes = 15)

In [179]:
n = 1
rows_to_drop = []
while n < df2.shape[0] - 1: 
    if (df2.loc[n-1, 'Zeitstempel Ende'] == df2.loc[n, 'Zeitstempel'] and df2.loc[n, 'Zeitstempel Ende'] == df2.loc[n + 1, 'Zeitstempel']):
        rows_to_drop.append(n)
    n += 1

In [180]:
df2 = df2.drop(rows_to_drop)

In [181]:
index = list(range(df2.shape[0]))
df2['Index'] = index
df2 = df2.set_index('Index')

In [194]:
n = 0
rows_to_drop = []
while n < df.shape[0]:
    m = 1
    #if n == 1800:
        #print(n)
    while m < df2.shape[0]:
        if (df.loc[n, 'Timestamp'] >= df2.loc[m - 1, 'Zeitstempel'] and df.loc[n, 'Timestamp'] < df2.loc[m, 'Zeitstempel Ende'] ):
            rows_to_drop.append(n)
            #if n > 1838:
                #print('row to drop: {}'.format(n))
                #print('m: {}'.format(m))
                #print('Time to drop: {}'.format(df.loc[n, 'Timestamp']))
            break
        elif m < df2.shape[0] - 1: 
            if (df.loc[n, 'Timestamp'] >= df2.loc[m, 'Zeitstempel Ende'] and df.loc[n, 'Timestamp'] < df2.loc[m + 1, 'Zeitstempel']):
            #if n > 1838:
                #print('loop was broken (n,m) = ({},{})'.format(n,m))
                #print('Time: {}'.format(df.loc[n, 'Timestamp']))
                break
        #print(m)
        #if n > 1838:
            #print('loop was not broken (n,m) = ({},{})'.format(n,m))
            #print('Time: {}'.format(df.loc[n, 'Timestamp']))
        m += 2
    #print(n)
    n += 1

In [195]:
df = df.drop(rows_to_drop)

In [197]:
#Indexkorrektur
index = list(range(df.shape[0]))
df['Index'] = index
df = df.set_index('Index')

In [146]:
#rows_to_drop

In [12]:
df3 = pd.read_excel(Bilanzieller_Ausgleich[0])

In [122]:
df2.head()

,Zeitstempel,Bilanzieller Ausgleich [MW],Bilanzieller Redispatch Global Tech I,Werte,Einheit,Zeitstempel Ende
Index,,,,,,
0,2024-01-13 12:00:00,10.726,NaN,NaN,NaN,2024-01-13 12:15:00
1,2024-01-13 17:45:00,18.868,NaN,NaN,NaN,2024-01-13 18:00:00
2,2024-01-13 18:30:00,0.992,NaN,NaN,NaN,2024-01-13 18:45:00
3,2024-01-14 08:45:00,43.326,NaN,NaN,NaN,2024-01-14 09:00:00
4,2024-01-14 21:00:00,37.208,NaN,NaN,NaN,2024-01-14 21:15:00


In [170]:
df2.to_excel('Bilanzieller Ausgleich.xlsx')

In [134]:
df.iloc[4463: 4467]

,Timestamp,Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten [kW] - GT01 Umrichter,Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten [kW] - GT02 Umrichter,Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten [kW] - GT03 Umrichter,Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten [kW] - GT04 Umrichter,Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten [kW] - GT05 Umrichter,Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten [kW] - GT06 Umrichter,Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten [kW] - GT07 Umrichter,Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten [kW] - GT08 Umrichter,Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten [kW] - GT09 Umrichter,...,Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten [kW] - GT71 Umrichter,Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten [kW] - GT72 Umrichter,Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten [kW] - GT73 Umrichter,Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten [kW] - GT74 Umrichter,Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten [kW] - GT75 Umrichter,Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten [kW] - GT76 Umrichter,Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten [kW] - GT77 Umrichter,Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten [kW] - GT78 Umrichter,Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten [kW] - GT79 Umrichter,Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten [kW] - GT80 Umrichter
Index,,,,,,,,,,,,,,,,,,,,,
4463,2024-01-31 23:50:00,-0.2651,3138.5,3077.7,3084.6,-0.4606,3129.1,NaN,-0.9183,3053.9,...,NaN,3405.1,3941.2,3965.2,3408.0,3917.6,3884.8,-0.5166,3979.0,3427.2
4464,2024-02-01 00:00:00,-0.2663,3138.6,3077.1,3084.3,-0.5426,3129.8,NaN,-0.9098,3054.7,...,NaN,3410.2,3939.8,3964.8,3407.9,3919.5,3883.8,-0.4518,3978.4,3428.4
4465,2024-02-01 00:10:00,-0.1738,3194.2,3132.3,3140.0,-0.4793,3185.4,NaN,-0.9071,3109.3,...,NaN,3484.7,3940.9,3964.9,3485.5,3918.4,3884.0,-0.3914,3977.5,3505.1
4466,2024-02-01 00:20:00,-0.1202,3386.2,3319.8,3329.1,-0.5258,3377.6,NaN,-0.9163,3301.3,...,NaN,3760.9,3940.0,3965.5,3770.3,3919.8,3884.9,-0.5617,3978.4,3792.7


In [55]:
if (df.loc[1800, 'Timestamp'] == df2.loc[1200, 'Zeitstempel'] and df2.loc[1200, 'Bilanzieller Ausgleich [MW]'] > 0):
    rows_to_drop.append(1800)

In [196]:
df.to_excel('10-Minuten Mittelwerte Anlagenleistung bereinigt.xlsx', index = False)

In [202]:
df_test.to_excel('10-Minuten Mittelwerte Anlagenleistung Test.xlsx', index = False)

In [58]:
df = pd.DataFrame()
for file in Anlagenleistung:
    df = pd.concat([df, pd.read_excel(file)], ignore_index = True)

In [59]:
df = df.drop_duplicates()

In [3]:
df_test = correlation_check(Anlagenleistung, Netzverfügbarkeit, Bilanzieller_Ausgleich)

FileNotFoundError: [Errno 2] No such file or directory: '../../Multiversum/2024/01-2024/power/10-min-Mittelwerte Anlagenleistung.xlsx'

In [203]:
df = df_test


In [123]:
df2.shape

(262, 6)

In [124]:
df2.tail()

,Zeitstempel,Bilanzieller Ausgleich [MW],Bilanzieller Redispatch Global Tech I,Werte,Einheit,Zeitstempel Ende
Index,,,,,,
257,2024-11-27 20:45:00,29.286,NaN,NaN,NaN,2024-11-27 21:00:00
258,2024-11-28 03:30:00,29.840,NaN,NaN,NaN,2024-11-28 03:45:00
259,2024-11-28 07:45:00,68.291,NaN,NaN,NaN,2024-11-28 08:00:00
260,2024-11-29 21:30:00,64.724,NaN,NaN,NaN,2024-11-29 21:45:00
261,2024-11-29 23:45:00,42.052,NaN,NaN,NaN,2024-11-30 00:00:00


In [15]:
df1 = pd.read_excel('Netzverfügbarkeiten.xlsx', header = 9)

In [198]:
df.head()

,Timestamp,Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten [kW] - GT01 Umrichter,Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten [kW] - GT02 Umrichter,Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten [kW] - GT03 Umrichter,Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten [kW] - GT04 Umrichter,Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten [kW] - GT05 Umrichter,Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten [kW] - GT06 Umrichter,Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten [kW] - GT07 Umrichter,Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten [kW] - GT08 Umrichter,Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten [kW] - GT09 Umrichter,...,Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten [kW] - GT71 Umrichter,Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten [kW] - GT72 Umrichter,Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten [kW] - GT73 Umrichter,Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten [kW] - GT74 Umrichter,Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten [kW] - GT75 Umrichter,Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten [kW] - GT76 Umrichter,Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten [kW] - GT77 Umrichter,Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten [kW] - GT78 Umrichter,Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten [kW] - GT79 Umrichter,Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten [kW] - GT80 Umrichter
Index,,,,,,,,,,,,,,,,,,,,,
0,2024-01-01 00:00:00,-0.2793,5242.9,-0.3255,4051.2,-0.5103,4112.8,NaN,4305.8,-0.5451,...,NaN,5225.0,3825.2,-0.5019,5195.6,3805.6,3752.1,-0.5233,3860.7,3720.8000
1,2024-01-01 00:10:00,-0.1851,5255.8,-0.2193,4052.6,-0.5060,4113.1,NaN,4315.8,-0.5317,...,NaN,5222.8,3829.5,-0.4579,5226.1,3803.6,3756.4,-0.4776,3868.4,-0.6722
2,2024-01-01 00:20:00,-0.1787,5248.6,-0.3450,4049.2,-0.4854,4111.6,NaN,4407.4,-0.4984,...,NaN,5200.2,3825.2,-0.4614,5157.8,3804.4,3774.4,-0.4396,3864.7,-0.8849
3,2024-01-01 00:30:00,-0.3095,5244.2,-0.3781,4054.2,-0.5191,4113.3,NaN,4610.5,-0.5448,...,NaN,5121.2,3827.9,-0.4975,4886.4,3803.1,3744.0,-0.5118,3868.2,-0.6838
4,2024-01-01 00:40:00,-0.1661,5249.9,-0.2527,4051.0,-0.4811,4112.4,NaN,4593.1,-0.5308,...,NaN,5226.6,3828.3,-0.4631,5050.9,3781.0,3780.8,-0.4279,3862.1,-0.9088


In [23]:
df1.drop([0,1])

,Unnamed: 0,Unnamed: 1,Abschaltzeit,Zuschaltung NEA,Freigabe zur Einspeisung von TenneT,GenSet ausgeschaltet,33kV zugeschaltet,Freigabe zur Einspeisung an Adwen,Geplant?,Ursache,NAP,Unnamed: 11,Dauer TenneT,Dauer Adwen,Unnamed: 14
2,NaN,3.0,2016-06-26 12:03:00,2016-06-26 14:58:00,2016-06-27 19:45:00,2016-06-27 21:29:00,2016-06-27 21:50:00,2016-06-27 22:13:00,ungeplant,Defekt auf der Borwin 2,NAP 1&2,NaN,1 days 07:42:00,1 days 10:10:00,NaN
3,NaN,4.0,2016-10-25 07:01:00,2016-10-25 07:28:00,2016-10-25 19:18:00,2016-10-25 19:51:00,2016-10-25 20:18:00,2016-10-25 20:19:00,geplant,Wartungsmaßnahme TenneT Borwin 2,NAP 1&2,NaN,0 days 12:17:00,0 days 13:18:00,NaN
4,NaN,5.0,2017-07-03 07:13:00,2017-07-03 07:41:00,2017-07-07 23:26:00,2017-07-08 00:32:00,2017-07-08 00:50:00,2017-07-08 08:42:00,geplant,Wartungsmaßnahme TenneT Borwin 2,NAP 1&2,NaN,4 days 16:13:00,5 days 01:29:00,NaN
5,NaN,6.0,2017-09-12 09:32:00,NaT,2017-09-12 10:53:00,NaT,2017-09-12 11:11:00,2017-09-12 11:24:00,geplant,NAP 2 für Wartung Tennet (kein NEA-Betrieb),NAP 2,NaN,0 days 00:40:30,0 days 01:52:00,NaN
6,NaN,7.0,2017-09-21 08:05:00,2017-09-21 07:25:00,2017-09-21 18:08:00,2017-09-21 18:53:00,2017-09-21 18:59:00,2017-09-21 19:34:00,geplant,Störung TenneT,NAP 1&2,NaN,0 days 10:03:00,0 days 11:29:00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,NaN,396.0,NaT,NaT,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaN
396,NaN,397.0,NaT,NaT,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaN
397,NaN,398.0,NaT,NaT,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaN
398,NaN,399.0,NaT,NaT,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaN


In [20]:
df.loc[0, 'Timestamp'] > df1.loc[0, 'Freigabe zur Einspeisung von TenneT']

True

In [55]:
n = 0
rows_to_drop = []
while n < df.shape[0]:
    m = 0
    while m < df1.shape[0]:
        if (df.loc[n, 'Timestamp'] >= df1.loc[m, 'Abschaltzeit'] and df.loc[n, 'Timestamp'] <= df1.loc[m, 'Freigabe zur Einspeisung von TenneT']):
            rows_to_drop.append(n)
            break
        elif (df.loc[n, 'Timestamp'] > df1.loc[m, 'Freigabe zur Einspeisung von TenneT'] and df.loc[n, 'Timestamp'] < df1.loc[m+1, 'Abschaltzeit']):
            break
        m += 1
    n += 1

df = df.drop(rows_to_drop)

In [32]:
rows_to_drop[-5:]

[]

In [61]:
df = pd.DataFrame()

In [63]:
for file in Anlagenleistung:
    df = pd.concat([df, pd.read_excel(file)], ignore_index = True)
    df = df.drop_duplicates()

In [54]:
df = pd.concat([df1, df2], ignore_index = True)

In [72]:
df = df.drop_duplicates()

In [64]:
df.dtypes

Timestamp                                                                     datetime64[ns]
Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten  [kW] - GT01 Umrichter           float64
Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten  [kW] - GT02 Umrichter           float64
Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten  [kW] - GT03 Umrichter           float64
Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten  [kW] - GT04 Umrichter           float64
                                                                                   ...      
Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten  [kW] - GT76 Umrichter           float64
Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten  [kW] - GT77 Umrichter           float64
Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten  [kW] - GT78 Umrichter           float64
Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten  [kW] - GT79 Umrichter           float64
Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten  [kW] - GT80 Umric

In [54]:
df.iloc[ 43958:43965]

,Timestamp,Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten [kW] - GT01 Umrichter,Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten [kW] - GT02 Umrichter,Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten [kW] - GT03 Umrichter,Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten [kW] - GT04 Umrichter,Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten [kW] - GT05 Umrichter,Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten [kW] - GT06 Umrichter,Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten [kW] - GT07 Umrichter,Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten [kW] - GT08 Umrichter,Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten [kW] - GT09 Umrichter,...,Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten [kW] - GT71 Umrichter,Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten [kW] - GT72 Umrichter,Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten [kW] - GT73 Umrichter,Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten [kW] - GT74 Umrichter,Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten [kW] - GT75 Umrichter,Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten [kW] - GT76 Umrichter,Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten [kW] - GT77 Umrichter,Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten [kW] - GT78 Umrichter,Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten [kW] - GT79 Umrichter,Wirkleistung Netz OS-seitig Mittelwert - 10 Minuten [kW] - GT80 Umrichter
Index,,,,,,,,,,,,,,,,,,,,,
43958,2024-11-01 05:50:00,NaN,-0.1958,-0.2621,-0.4986,-0.4286,-0.4143,-0.9424,-0.7146,-0.5604,...,0.0,0.0,-0.1586,-0.4606,-0.3173,-0.2341,-0.6295,0.0,-0.7455,-0.7448
43959,2024-11-01 06:00:00,NaN,-0.2026,-0.2495,-0.5038,-0.4181,-0.4131,-0.9426,-0.7455,-0.5506,...,0.0,0.0,-0.1525,-0.4593,-0.3050,-0.2238,-0.6178,0.0,-0.7323,-0.7512
43960,2024-11-01 06:10:00,NaN,-0.2021,-0.2731,-0.5093,-0.4151,-0.4210,-0.9446,-0.7541,-0.5508,...,0.0,0.0,-0.1580,-0.4620,-0.3224,-0.2290,-0.6140,0.0,-0.7252,-0.7788
43961,2024-11-01 06:20:00,NaN,-0.1883,-0.2727,-0.5008,-0.4276,-0.4129,-0.9415,-0.7335,-0.5261,...,0.0,0.0,-0.1272,-0.4686,-0.2931,-0.2476,-0.6253,0.0,-0.7164,-0.6262
43962,2024-11-01 06:30:00,NaN,-0.1883,-0.2500,-0.4973,-0.4275,-0.4068,-0.9049,-0.7315,-0.5040,...,0.0,0.0,-0.0956,-0.4687,-0.3089,-0.2784,-0.6340,0.0,-0.7212,-0.8248
43963,2024-11-01 06:40:00,NaN,-0.1876,-0.2280,-0.1675,-0.1418,-0.1253,-0.3291,-0.2253,-0.5161,...,0.0,0.0,-0.1026,-0.4758,-0.2961,-0.2800,-0.6328,0.0,-0.7105,-0.8268
43964,2024-11-01 06:50:00,NaN,-0.1783,-0.2644,0.0000,0.0000,0.0000,0.0000,0.0000,-0.5034,...,0.0,0.0,-0.1061,-0.4740,-0.3023,-0.2797,-0.6265,0.0,-0.7150,-0.7833


In [39]:
df1.shape

(4465, 81)

In [199]:
df.shape

(17048, 81)

In [68]:
df2.shape

(3107, 5)